In [7]:
import os
import glob
import pandas as pd
import textgrids
import tqdm
from utils import flac_to_wav

# Create Subset json

## Keep files in subset

In [15]:
import ast

def get_subset_df(json_dir=None):
    if not json_dir:
        json_dir = '/home/knoriy/Documents/laion/split_peoples_speech/flac_train_manifest.jsonl'

    with open(json_dir, 'r') as json_file:
        json_list = list(json_file)

    df = pd.DataFrame.from_dict(json_list)
    df[0] = df[0].apply(lambda x: ast.literal_eval(x))

    for keys in df[0][0].keys():
        df[keys] = [path[0][keys] for path in df.iloc()]

    subset = glob.glob('/home/knoriy/Documents/laion/split_peoples_speech/mini_subset/**/*.flac', recursive=True)
    subset = [os.path.join(*(dir.split(os.path.sep)[7:])) for dir in subset]

    subset_df = df[df['audio_filepath'].isin(subset)].reset_index(drop=True)
    subset_df = subset_df.drop(0, axis=1)

    return subset_df

df = get_subset_df()
df.to_csv('/home/knoriy/Documents/laion/split_peoples_speech/mini_subset.tsv', sep='\t', header=None, index=False)

# prepare MFA

In [32]:
# Clean old wavs ONLY FOR TESTING
import glob

old_wav_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/mini_subset/**/*.wav', recursive=True)
for wav_path in old_wav_path:
    os.remove(wav_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset_split/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# Tar loader

# ToDo
- [ ] Download and extract single or batch of files from aws
- [ ] Process for MFA
- [ ] Generate textgrid
- [ ] Split audio and upload to aws s3

In [1]:
import tarfile
import shutil
from main import *
from utils import generate_txt

import warnings

In [2]:
filename = '/home/knoriy/Documents/laion/split_peoples_speech/mini_subset.tar.xz'
base_pps_dataset_path = '/home/knoriy/Documents/laion/split_peoples_speech/'
chunk = 5


In [3]:
with tarfile.open(filename,'r') as file_obj:
    file_names_full_list = file_obj.getnames()

    for i in range(0, len(file_names_full_list), chunk):
        for file_name in file_names_full_list[i:i + chunk]:
            file_obj.extract(file_name, './')

        generate_subset_tsv = True
        if generate_subset_tsv == True:
            df = get_subset_df().to_csv(f'/home/knoriy/Documents/laion/split_peoples_speech/subset_{i}.tsv', sep='\t', header=None, index=False)
        # Save transcript to file
        # txt_path = os.path.join(base_pps_dataset_path, f'{file_name.split(".")[0]}.txt')
        # generate_txt(txt_path, "Temp transcript")
        # warnings.warn('transcription is using temp text')
        # save_all_text_to_file(df)

        # Convert Flac to wav
        # flac_path = os.path.join(base_pps_dataset_path, file_name)
        # wav_path = os.path.join(base_pps_dataset_path, f'{file_name.split(".")[0]}.wav')
        # flac_to_wav(file_name, wav_path, overwrite=True)
        # convert_all_to_wav(df, base_pps_dataset_path)

        # generate_textgrids(os.path.join(base_pps_dataset_path, 'mini_subset'))
        # shutil.rmtree('/home/knoriy/Documents/laion/split_peoples_speech/mini_subset')

                                          audio_filepath  duration  shard_id  \
0      Peters_Twp_DOT__School_Board_-_Policy_Committe...     14.89        -1   
1      Peters_Twp_DOT__School_Board_-_Policy_Committe...      6.24        -1   
2      Peters_Twp_DOT__School_Board_-_Policy_Committe...     14.86        -1   
3      Peters_Twp_DOT__School_Board_-_Policy_Committe...     14.68        -1   
4      Peters_Twp_DOT__School_Board_-_Finance_and_Bui...     14.95        -1   
...                                                  ...       ...       ...   
46075           mnn_514_4914/514_4914_DOT_mp3_00006.flac     11.64        -1   
46076           mnn_514_4882/514_4882_DOT_mp3_00025.flac     14.82        -1   
46077           mnn_514_4885/514_4885_DOT_mp3_00006.flac     14.86        -1   
46078           mnn_514_4788/514_4788_DOT_mp3_00015.flac     15.00        -1   
46079           mnn_514_4904/514_4904_DOT_mp3_00019.flac     14.82        -1   

                                       

KeyboardInterrupt: 

In [11]:
with tarfile.open(filename,'r') as file_obj:
    file_names_full_list = file_obj.members()
    print(file_names_full_list)

          0
0  0.135243
1  0.819354
2  0.020243
          0
0  0.856007
1  0.831876
2  0.344182
          0
0  0.017545
1  0.616152
2  0.029083


# AWS S3

In [ ]:
%%bash

nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-07a8f0d3-6d27-4299-887a-dc12a6d72f8d-c000.tar?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A30%3A22Z/-1/host/444b9c082ceffd10f38bb965679ed9ec12202836831e111dd193fde281062d26 -O - | aws s3 cp - s3://s-laion/peoples_speech/train_clean_pps.tar" &;
nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-4e132642-c01c-4db6-9db0-a1e19193f6f8-c000.json?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A31%3A22Z/-1/host/d7dacf3c31d2e3670d82727636ce234be27a9128df7a80883b84b4a3d8c7f6c0 -O - | aws s3 cp - s3://s-laion/peoples_speech/Manifest.json" &;


In [ ]:
%%bash

nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &



In [ ]:
%%bash

# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &



In [3]:
import os
os.system('pwd')

/home/knoriy/Documents/laion/split_peoples_speech


0

In [39]:
import glob
import tarfile

files = glob.glob('/home/knoriy/Documents/laion/split_peoples_speech/mini_subset_split/**/*.*')

with tarfile.open('/home/knoriy/Documents/laion/split_peoples_speech/test.zip', mode='a') as src_file_obj:
    for file in files:
        src_file_obj.add(file)

ReadError: invalid header

In [30]:
with tarfile.open('/home/knoriy/Documents/laion/split_peoples_speech/test.tar.xz', mode='w|xz') as src_file_obj:
    src_file_obj.add('/home/knoriy/Documents/laion/split_peoples_speech/utils.py')